In [3]:
### Plotting bettis

script_start_time = time()
println("\nimporting packages...")

using Pkg
using Statistics
using LinearAlgebra
using Eirene
using StatsBase
using JLD
using Plots
Pkg.add("ColorSchemes")
using ColorSchemes
using JSON

include("helper_functions.jl")
println("packages and functions imported")
printstyled("Elapsed time = $(time() - script_start_time) seconds \n \n", color = :yellow)


importing packages...
 Resolving package versions...
  Updating `/opt/julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.2/Manifest.toml`
 [no changes]
packages and functions imported
Elapsed time = 0.32238197326660156 seconds 
 


In [4]:
### Set parameters

# Read from config file
config_file = "config091720.json"
config = read_config("$(homedir())/configs/$(config_file)")

# Parameters for all graphs
const NNODES = config["NNODES"]
const MAXDIM = config["MAXDIM"]
const NREPS = config["NREPS"]
const DATE_STRING = config["DATE_STRING"]


### Locate data and find nametags
read_dir = "../processed_data/results/$(NNODES)nodes"
betti_files = filter(x->occursin("_bettis.jld",x), readdir(read_dir))
betti_files = filter(x->occursin(DATE_STRING, x), betti_files)
println("Located the following graph files:")
for betti_file in betti_files
    println(betti_file)
end


# Locate the nametags
nametags = []
for betti_file in betti_files
    println(betti_file)
    tag = split(split(betti_file, "$(DATE_STRING)_")[2], "_bettis")[1]
    nametags = [nametags; tag]
end

nametags = unique(nametags)


Located the following graph files:
IID_70nodes_100reps_091720_thresh02_edge483_noiseOnly_bettis.jld
IID_70nodes_100reps_091720_thresh02_edge483_threshold_bettis.jld
IID_70nodes_100reps_091720_thresh03_edge725_noiseOnly_bettis.jld
IID_70nodes_100reps_091720_thresh03_edge725_threshold_bettis.jld
IID_70nodes_100reps_091720_thresh04_edge966_noiseOnly_bettis.jld
IID_70nodes_100reps_091720_thresh04_edge966_threshold_bettis.jld
IID_70nodes_100reps_091720_thresh05_edge1208_noiseOnly_bettis.jld
IID_70nodes_100reps_091720_thresh05_edge1208_threshold_bettis.jld
IID_70nodes_100reps_091720_thresh06_edge1449_noiseOnly_bettis.jld
IID_70nodes_100reps_091720_thresh06_edge1449_threshold_bettis.jld
IID_70nodes_100reps_091720_thresh07_edge1691_noiseOnly_bettis.jld
IID_70nodes_100reps_091720_thresh07_edge1691_threshold_bettis.jld
IID_70nodes_100reps_091720_thresh08_edge1932_noiseOnly_bettis.jld
IID_70nodes_100reps_091720_thresh08_edge1932_threshold_bettis.jld
assortative_70nodes_100reps_20_5_10_5_091720_th

probTriangle1_70nodes_100reps_01_091720_thresh02_edge483_threshold_bettis.jld
probTriangle1_70nodes_100reps_01_091720_thresh03_edge725_noiseOnly_bettis.jld
probTriangle1_70nodes_100reps_01_091720_thresh03_edge725_threshold_bettis.jld
probTriangle1_70nodes_100reps_01_091720_thresh04_edge966_noiseOnly_bettis.jld
probTriangle1_70nodes_100reps_01_091720_thresh04_edge966_threshold_bettis.jld
probTriangle1_70nodes_100reps_01_091720_thresh05_edge1208_noiseOnly_bettis.jld
probTriangle1_70nodes_100reps_01_091720_thresh05_edge1208_threshold_bettis.jld
probTriangle1_70nodes_100reps_01_091720_thresh06_edge1449_noiseOnly_bettis.jld
probTriangle1_70nodes_100reps_01_091720_thresh06_edge1449_threshold_bettis.jld
probTriangle1_70nodes_100reps_01_091720_thresh07_edge1691_noiseOnly_bettis.jld
probTriangle1_70nodes_100reps_01_091720_thresh07_edge1691_threshold_bettis.jld
probTriangle1_70nodes_100reps_01_091720_thresh08_edge1932_noiseOnly_bettis.jld
probTriangle1_70nodes_100reps_01_091720_thresh08_edge1932

probTriangle_70nodes_100reps_1_091720_thresh05_edge1208_noiseOnly_bettis.jld
probTriangle_70nodes_100reps_1_091720_thresh05_edge1208_threshold_bettis.jld
probTriangle_70nodes_100reps_1_091720_thresh06_edge1449_noiseOnly_bettis.jld
probTriangle_70nodes_100reps_1_091720_thresh06_edge1449_threshold_bettis.jld
probTriangle_70nodes_100reps_1_091720_thresh07_edge1691_noiseOnly_bettis.jld
probTriangle_70nodes_100reps_1_091720_thresh07_edge1691_threshold_bettis.jld
probTriangle_70nodes_100reps_1_091720_thresh08_edge1932_noiseOnly_bettis.jld
probTriangle_70nodes_100reps_1_091720_thresh08_edge1932_threshold_bettis.jld
randomGeom_70nodes_100reps_3_091720_thresh02_edge483_noiseOnly_bettis.jld
randomGeom_70nodes_100reps_3_091720_thresh02_edge483_threshold_bettis.jld
randomGeom_70nodes_100reps_3_091720_thresh03_edge725_noiseOnly_bettis.jld
randomGeom_70nodes_100reps_3_091720_thresh03_edge725_threshold_bettis.jld
randomGeom_70nodes_100reps_3_091720_thresh04_edge966_noiseOnly_bettis.jld
randomGeom_70n

disassortative_70nodes_100reps_20_5_10_5_091720_thresh08_edge1932_threshold_bettis.jld
discreteUniformConf_70nodes_100reps_0_1000_091720_thresh02_edge483_noiseOnly_bettis.jld
discreteUniformConf_70nodes_100reps_0_1000_091720_thresh02_edge483_threshold_bettis.jld
discreteUniformConf_70nodes_100reps_0_1000_091720_thresh03_edge725_noiseOnly_bettis.jld
discreteUniformConf_70nodes_100reps_0_1000_091720_thresh03_edge725_threshold_bettis.jld
discreteUniformConf_70nodes_100reps_0_1000_091720_thresh04_edge966_noiseOnly_bettis.jld
discreteUniformConf_70nodes_100reps_0_1000_091720_thresh04_edge966_threshold_bettis.jld
discreteUniformConf_70nodes_100reps_0_1000_091720_thresh05_edge1208_noiseOnly_bettis.jld
discreteUniformConf_70nodes_100reps_0_1000_091720_thresh05_edge1208_threshold_bettis.jld
discreteUniformConf_70nodes_100reps_0_1000_091720_thresh06_edge1449_noiseOnly_bettis.jld
discreteUniformConf_70nodes_100reps_0_1000_091720_thresh06_edge1449_threshold_bettis.jld
discreteUniformConf_70nodes_1

probTriangle6_70nodes_100reps_06_091720_thresh03_edge725_noiseOnly_bettis.jld
probTriangle6_70nodes_100reps_06_091720_thresh03_edge725_threshold_bettis.jld
probTriangle6_70nodes_100reps_06_091720_thresh04_edge966_noiseOnly_bettis.jld
probTriangle6_70nodes_100reps_06_091720_thresh04_edge966_threshold_bettis.jld
probTriangle6_70nodes_100reps_06_091720_thresh05_edge1208_noiseOnly_bettis.jld
probTriangle6_70nodes_100reps_06_091720_thresh05_edge1208_threshold_bettis.jld
probTriangle6_70nodes_100reps_06_091720_thresh06_edge1449_noiseOnly_bettis.jld
probTriangle6_70nodes_100reps_06_091720_thresh06_edge1449_threshold_bettis.jld
probTriangle6_70nodes_100reps_06_091720_thresh07_edge1691_noiseOnly_bettis.jld
probTriangle6_70nodes_100reps_06_091720_thresh07_edge1691_threshold_bettis.jld
probTriangle6_70nodes_100reps_06_091720_thresh08_edge1932_noiseOnly_bettis.jld
probTriangle6_70nodes_100reps_06_091720_thresh08_edge1932_threshold_bettis.jld
probTriangle7_70nodes_100reps_07_091720_thresh02_edge483

14-element Array{Any,1}:
 "thresh02_edge483_noiseOnly" 
 "thresh02_edge483_threshold" 
 "thresh03_edge725_noiseOnly" 
 "thresh03_edge725_threshold" 
 "thresh04_edge966_noiseOnly" 
 "thresh04_edge966_threshold" 
 "thresh05_edge1208_noiseOnly"
 "thresh05_edge1208_threshold"
 "thresh06_edge1449_noiseOnly"
 "thresh06_edge1449_threshold"
 "thresh07_edge1691_noiseOnly"
 "thresh07_edge1691_threshold"
 "thresh08_edge1932_noiseOnly"
 "thresh08_edge1932_threshold"

In [8]:


for (j,nametag) in enumerate(nametags)
    
    ## Create one large plot with all of the betti curves
    betti_files_nametag = filter(x -> occursin("$(nametag)",x), betti_files)
    model_names = [split(betti_file_nametag,"_")[1] for betti_file_nametag in betti_files_nametag]
    
    nModels = length(betti_files_nametag)
    nEdges = binomial(NNODES, 2)
    bettisAll = zeros(NREPS, nEdges, MAXDIM, nModels)
    
    for (i,betti_file_nametag) in enumerate(betti_files_nametag)

        # Read in Betti curves
        betti_dict = load("$(read_dir)/$(betti_file_nametag)")
        bettisAll[:,:,:, i] = betti_dict["bettisArray"]
        println(betti_file_nametag)

    end

    bettisAllAvg = dropdims(mean(bettisAll, dims=1), dims=1)
    println("Size bettisAllAvg is $(size(bettisAllAvg))")

    # Finally plot
    p1a = plot(bettisAllAvg[:,1,:], layout = 24, size = (2000,1600), title = ["$(model_names[i])" for j=1:1, i=1:nModels])
    for d=2:MAXDIM
        plot!(bettisAllAvg[:,d,:],legend = nothing, xlabel = "Edges added", ylabel="Beta_k")

    end
    
    # If the file is of threshold type, add the threshold line
    if occursin("threshold", nametag)
        
        # Get threshold edge
        thresh_string = split(split(nametag,"edge")[2],"_")[1]
        threshold_edge = parse(Int, thresh_string)
        println("processing threshold edge $(threshold_edge)")
        
        # Plot the vertical line
        vline!(transpose([threshold_edge for i=1:nModels]), color=:gray)
#         yaxis!([zeros(1,length(yaxis_lims)), yaxis_lims])
    end


    savefig("../figures/all_bettis_$(nametag)_$(DATE_STRING).pdf")
end






IID_70nodes_100reps_091720_thresh02_edge483_noiseOnly_bettis.jld
assortative_70nodes_100reps_20_5_10_5_091720_thresh02_edge483_noiseOnly_bettis.jld
coreperiphery_70nodes_100reps_15_5_10_5_091720_thresh02_edge483_noiseOnly_bettis.jld
cosineGeometric_70nodes_100reps_3_091720_thresh02_edge483_noiseOnly_bettis.jld
disassortative_70nodes_100reps_20_5_10_5_091720_thresh02_edge483_noiseOnly_bettis.jld
discreteUniformConf_70nodes_100reps_0_1000_091720_thresh02_edge483_noiseOnly_bettis.jld
dotProduct_70nodes_100reps_3_091720_thresh02_edge483_noiseOnly_bettis.jld
geometricConf_70nodes_100reps_001_100_091720_thresh02_edge483_noiseOnly_bettis.jld
probTriangle0_70nodes_100reps_0_091720_thresh02_edge483_noiseOnly_bettis.jld
probTriangle1_70nodes_100reps_01_091720_thresh02_edge483_noiseOnly_bettis.jld
probTriangle2_70nodes_100reps_02_091720_thresh02_edge483_noiseOnly_bettis.jld
probTriangle3_70nodes_100reps_03_091720_thresh02_edge483_noiseOnly_bettis.jld
probTriangle4_70nodes_100reps_04_091720_thresh

probTriangle3_70nodes_100reps_03_091720_thresh04_edge966_noiseOnly_bettis.jld
probTriangle4_70nodes_100reps_04_091720_thresh04_edge966_noiseOnly_bettis.jld
probTriangle5_70nodes_100reps_05_091720_thresh04_edge966_noiseOnly_bettis.jld
probTriangle6_70nodes_100reps_06_091720_thresh04_edge966_noiseOnly_bettis.jld
probTriangle7_70nodes_100reps_07_091720_thresh04_edge966_noiseOnly_bettis.jld
probTriangle8_70nodes_100reps_08_091720_thresh04_edge966_noiseOnly_bettis.jld
probTriangle9_70nodes_100reps_09_091720_thresh04_edge966_noiseOnly_bettis.jld
probTriangle_70nodes_100reps_1_091720_thresh04_edge966_noiseOnly_bettis.jld
randomGeom_70nodes_100reps_3_091720_thresh04_edge966_noiseOnly_bettis.jld
ringLattice_70nodes_100reps_091720_thresh04_edge966_noiseOnly_bettis.jld
rmsd_70nodes_100reps_3_091720_thresh04_edge966_noiseOnly_bettis.jld
sqdEuclidean_70nodes_100reps_3_091720_thresh04_edge966_noiseOnly_bettis.jld
Size bettisAllAvg is (2415, 4, 23)
IID_70nodes_100reps_091720_thresh04_edge966_threshol

sqdEuclidean_70nodes_100reps_3_091720_thresh06_edge1449_noiseOnly_bettis.jld
Size bettisAllAvg is (2415, 4, 23)
IID_70nodes_100reps_091720_thresh06_edge1449_threshold_bettis.jld
assortative_70nodes_100reps_20_5_10_5_091720_thresh06_edge1449_threshold_bettis.jld
coreperiphery_70nodes_100reps_15_5_10_5_091720_thresh06_edge1449_threshold_bettis.jld
cosineGeometric_70nodes_100reps_3_091720_thresh06_edge1449_threshold_bettis.jld
disassortative_70nodes_100reps_20_5_10_5_091720_thresh06_edge1449_threshold_bettis.jld
discreteUniformConf_70nodes_100reps_0_1000_091720_thresh06_edge1449_threshold_bettis.jld
dotProduct_70nodes_100reps_3_091720_thresh06_edge1449_threshold_bettis.jld
geometricConf_70nodes_100reps_001_100_091720_thresh06_edge1449_threshold_bettis.jld
probTriangle0_70nodes_100reps_0_091720_thresh06_edge1449_threshold_bettis.jld
probTriangle1_70nodes_100reps_01_091720_thresh06_edge1449_threshold_bettis.jld
probTriangle2_70nodes_100reps_02_091720_thresh06_edge1449_threshold_bettis.jld
p

probTriangle0_70nodes_100reps_0_091720_thresh08_edge1932_threshold_bettis.jld
probTriangle1_70nodes_100reps_01_091720_thresh08_edge1932_threshold_bettis.jld
probTriangle2_70nodes_100reps_02_091720_thresh08_edge1932_threshold_bettis.jld
probTriangle3_70nodes_100reps_03_091720_thresh08_edge1932_threshold_bettis.jld
probTriangle4_70nodes_100reps_04_091720_thresh08_edge1932_threshold_bettis.jld
probTriangle5_70nodes_100reps_05_091720_thresh08_edge1932_threshold_bettis.jld
probTriangle6_70nodes_100reps_06_091720_thresh08_edge1932_threshold_bettis.jld
probTriangle7_70nodes_100reps_07_091720_thresh08_edge1932_threshold_bettis.jld
probTriangle8_70nodes_100reps_08_091720_thresh08_edge1932_threshold_bettis.jld
probTriangle9_70nodes_100reps_09_091720_thresh08_edge1932_threshold_bettis.jld
probTriangle_70nodes_100reps_1_091720_thresh08_edge1932_threshold_bettis.jld
randomGeom_70nodes_100reps_3_091720_thresh08_edge1932_threshold_bettis.jld
ringLattice_70nodes_100reps_091720_thresh08_edge1932_thresh

In [11]:
##### Make special figures for thresholded stuff

# Maybe just betti bar first

# Filter to just the thresholds
thresh_files = filter(x->occursin("thresh",x), betti_files)
thresh_files = filter(x->!occursin("noiseOnly",x), thresh_files)

# Locate the thresh nametages
thresh_nametags = []
for thresh_file in thresh_files
    tag = split(split(thresh_file, "$(DATE_STRING)_")[2], "_bettis")[1]
    thresh_nametags = [thresh_nametags; tag]
end

thresh_nametags = unique(thresh_nametags)

## And locate the model names
model_names = []
for thresh_file in thresh_files
    name = split(thresh_file, "_thresh")[1]
    model_names = [model_names; name]
end

model_names = unique(model_names)


23-element Array{Any,1}:
 "IID_70nodes_100reps_091720"                       
 "assortative_70nodes_100reps_20_5_10_5_091720"     
 "coreperiphery_70nodes_100reps_15_5_10_5_091720"   
 "cosineGeometric_70nodes_100reps_3_091720"         
 "disassortative_70nodes_100reps_20_5_10_5_091720"  
 "discreteUniformConf_70nodes_100reps_0_1000_091720"
 "dotProduct_70nodes_100reps_3_091720"              
 "geometricConf_70nodes_100reps_001_100_091720"     
 "probTriangle0_70nodes_100reps_0_091720"           
 "probTriangle1_70nodes_100reps_01_091720"          
 "probTriangle2_70nodes_100reps_02_091720"          
 "probTriangle3_70nodes_100reps_03_091720"          
 "probTriangle4_70nodes_100reps_04_091720"          
 "probTriangle5_70nodes_100reps_05_091720"          
 "probTriangle6_70nodes_100reps_06_091720"          
 "probTriangle7_70nodes_100reps_07_091720"          
 "probTriangle8_70nodes_100reps_08_091720"          
 "probTriangle9_70nodes_100reps_09_091720"          
 "probTriangle_70node

In [17]:
nEdges = binomial(NNODES, 2)
plot_array3 = Any[]
plot_array4 = Any[]
for (j,model_name) in enumerate(model_names)
# model_name = "assortative_70nodes_50reps_15_5_10_5_091520"
    bettis = []

    model = split(model_name, "_")[1]
    # Loop through threshold vals and create vector of means and begin plotting

    betti_peaks_all = zeros(NREPS, MAXDIM, length(thresh_nametags))

    p3 = plot(layout = (MAXDIM, 1))

    rho_vec = []
    max_val_k =[]
    mean_bettis = []
    max_locs = Array{Float64}(undef, (0,MAXDIM))
    max_vals = Array{Float64}(undef, (0,MAXDIM))

    
    for (i,thresh_nametag) in enumerate(thresh_nametags)
        
        # Load data
        bettis_dict = load("$(read_dir)/$(model_name)_$(thresh_nametag)_bettis.jld")
        bettis = bettis_dict["bettisArray"]
        
    
        rho = split(split(thresh_nametag, "thresh")[2], "_")[1]
        rho = replace(rho, "0" => "0.")
        rho = parse(Float64, rho)
        
        rho_vec = [rho_vec; rho]
        
        mean_bettis = dropdims(mean(bettis, dims=1), dims=1)
    
        # Find max and location of max peak
        max_val_k = dropdims(maximum(mean_bettis, dims=1), dims=1)
        max_vals = [max_vals; transpose(max_val_k)]
    
        # Find locations of max value
        max_loc = zeros(1,4)
        for dim in collect(1:MAXDIM)
            max_loc[1, dim] = findfirst(mean_bettis[:, dim].==max_val_k[dim])
        end
        max_locs = [max_locs; max_loc]
            
        

    
        plot!([1:nEdges],mean_bettis, c=[[:blue] [:green] [:red] [:pink]], alpha = (rho+0.1), legend =false, lw=2)
#         xlabel!("Edges")
        ylabel!("Betti_k")

        
        
        
    end
    
    
    p4 = plot(size=(500,300))
    plot!(max_locs, max_vals, alpha = rho_vec, c=[[:blue] [:green] [:red] [:pink]],
        legend = false, m=:dot, lw=3, xlims=[0,nEdges])
    xlabel!("Peak Location")
    ylabel!("Peak Value")
    title!("$(model)")


        
#     plot(p3,p4,layout=(2, 1), size=(500,1000))
    push!(plot_array4,p4)
    push!(plot_array3,p3)


    
    
    
end

plot(plot_array4..., size = (3000,2000))
savefig("../figures/all_peakplots_$(DATE_STRING).pdf")


plot(plot_array3..., size = (1500,3000))
savefig("../figures/all_bettisthreshs_$(DATE_STRING).pdf")